In [2]:
import keras

Using TensorFlow backend.


In [3]:
keras.__version__

'2.2.4'

# Implementing character-level LSTM text generation
Let's put these ideas in practice in a Keras implementation. The first thing we need is a lot of text data that we can use to learn a language model. You could use any sufficiently large text file or set of text files -- Wikipedia, the Lord of the Rings, etc. In this example we will use some of the writings of Nietzsche, the late-19th century German philosopher (translated to English). The language model we will learn will thus be specifically a model of Nietzsche's writing style and topics of choice, rather than a more generic model of the English language.


In [4]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


In [5]:
maxlen=60
step=3
sentences=[]
next_chars=[]
for i in range(0,len(text)-maxlen,step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
print(len(sentences))

chars=sorted(list(set(text)))
print(len(chars))
char_indices=dict((char,chars.index(char))for char in chars)
x=np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y=np.zeros((len(sentences),len(chars)),dtype=np.bool)
for i,sentence in enumerate(sentences):
    for t,char in enumerate(sentence):
        x[i,t,char_indices[char]]=1
    y[i,char_indices[next_chars[i]]]=1

200278
58


In [6]:
from keras import layers

In [7]:
from keras.layers import LSTM,Dense
from keras.models import Sequential


In [8]:
model = Sequential()
model.add(LSTM(128,input_shape=(maxlen,len(chars))))
model.add(Dense(len(chars),activation='softmax'))

In [9]:
model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.RMSprop(lr=0.01))

In [10]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Given a trained model and a seed text snippet, we generate new text by repeatedly:

1) Drawing from the model a probability distribution over the next character given the text available so far
2) Reweighting the distribution to a certain "temperature"
3) Sampling the next character at random according to the reweighted distribution
4) Adding the new character at the end of the available text
This is the code we use to reweight the original probability distribution coming out of the model, and draw a character index from it (the "sampling function"):

In [11]:
import random
import sys
for epoch in range(1,60):
    print('epoch',epoch)
    model.fit(x,y,batch_size=128,epochs=1)
    start_index=random.randint(0,len(text)-maxlen-1)
    generated_text=text[start_index:start_index+maxlen]

    for temperature in [0.2,0.5,1.0,1.2]:
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled=np.zeros((1,maxlen,len(chars)))
            for t,char in enumerate(generated_text):
                sampled[0,t,char_indices[char]]=1.
            preds=model.predict(sampled,verbose=0)[0]
            next_index=sample(preds,temperature)
            next_char=chars[next_index]
            generated_text+=next_char
            generated_text=generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush
            
        print()

epoch 1
Epoch 1/1
200278/200278 [==============================] - 77s 387us/step - loss: 1.9831
s as fully established). i have no doubt that as men
argue indistent and religious of the stind of the strong and the stind to the sense of the spirit of the conscience of the spirit of the strong to the spirit of the sense of the sense of the spirit and the string the spirit and sensience and such the strong and in the strong in the string to the condition of the sense of the strong and the sense of the streng the spirit and compreself and the streng the 
 of the streng the spirit and compreself and the streng the fund and the long the free and his has probe of the speciently and original wit wish and the ore and at one in the probant only and religing that the meliengs of experience, and noble--and sense of conception of the comanity of the refuling to "not far even bo disfere how sense the destines in enery and stand and even be doous with experience the revered indelist of the spirit do

osophers--there is no other alternative: in minds strong and strength and the power, and who have to have to the power of the self-consideration of the pain of the strength and man as a such a such a such a such a superioring to the power, and the power of the countred and the sublimity of the power of the consideration of the presented and superficial and the experience of the complication of the presence of the self-contrary of the consideration of the s
 presence of the self-contrary of the consideration of the sense of all desires to happiness of the accally difference of the superfect of the profound of all the subtle and man, and impulse of the excepted that the faculity in the present that every something his false and good and whole says of the substrianed, and say
and the consideration of men in opposite the most fanishents of the experience
and pruth conduct of the exceptions the temptation of a co
e
and pruth conduct of the exceptions the temptation of a conception. he eltea

organical and the spirit and the spiritual states of the subliment and sure and the same the spirit and the consequently and the still himself and desire and the spirit and the states of the spirit to the spirit and the present the spirit and such a consequently and some something them as the consequently and the spirit of the spiritual and the spirit and the strength and some such a profound the sta
the spirit and the strength and some such a profound the states and deself when
the scientific standard of the chirce of the superation of his places and something subjuger and surparting in present man is a more one in the world of the sprint of the spirit and perhaps and instinct of all the consequently of the good point of the eloquent
emotion an action of distrust and all in the spiritual in distingle without of the raged to philosophy and soul, and mi
istingle without of the raged to philosophy and soul, and mind than
is he severity,
some love consist estieate that and dignable feedin

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


riod shor
itics of the will agisisagnority. eirraciw of in
period shorte
taste.
                                     nothinitys,
and peed that (all, forms
of
the exochardens supined not
"from
tressugh hiv). why relic in the everis formed
a
disposince, but conduct, who onemy, "tikons, discours
byfore, some light to
me theregese formmednonced ovanhy, onlom of
subject of the ceverial,
favitoulhed his make desirts for  wherever, actions something
nive certain obalint th
epoch 11
Epoch 1/1
200278/200278 [==============================] - 74s 368us/step - loss: 1.3917
 think
that all our feelings and doings are, at bottom, acts of the conception of the soul of the moral the super-include the super-inception of the account of the sentiments of the conceptions and the such a soul of the spirit of the specialty of the states of the sentiments and self-concerning the profound and self-absolutely and and the moral something to be a common the subject of the soul of the most conception of the actu

...

dual him. we are spirit of an instinct power. was moneter and point for cunusagincy? that icked in the value in juddrence of any vengitainately--avoulde; they tskind,
brtome spor teleed, and welferant" and books of this
a.
every
viols in
it
very
may be enough spouls an deepens natures to conscient new
as had painable, do religiors:
nlaw nature thmorfolent med as usubder-or life,
literiest faith" (ands at carust uncase mack. reliec from what delusision it f
epoch 40
Epoch 1/1
200278/200278 [==============================] - 81s 402us/step - loss: 1.3269
eedy, inhuman, avaricious, be he noble or servile; bad
is the sense of the soul and the sense of the morality of the spirit of the sense of the same to the process of the end of the sense of the demands the spirit of the spirit and sense of the same the superstition of the
sense of the superstition of the consensuagal proved and soul and the strength of the spirit of the superficial strength to the same to the strength of the morality of

get it faising it now that
glide
epoch 44
Epoch 1/1
200278/200278 [==============================] - 78s 390us/step - loss: 1.3476
 psychology, a certain kind of imaginativeness
in the interpretation of the spirit of the strength of the spirit of the self distrust and the moral belief of the soul and self-saint and self the more and something which the spirit of the spirit of the fact of the spirit of the power of the strength of the spirit of the contempt of the most existence of the strength of the contempt of the strength of the strength of the strength of the strength of the power
of the strength of the strength of the strength of the powerful something which the spirit of the world to the foregres of the man to the world for the belief of the atcernce of something and the higher by the world in the stories of the dangers of question of there
is always to
the saint of these maft, the
last and self the higher man, and to do a litile the ordingengance) of the soul, and element and en

but now when it has been a desires to the sense of the more the more and such a man and soul and soul and soul and soul and such a conscience to be a soul and self-content and the conscience of the action of the storn of the sense of the more the self-content of the sense of the such and soul. the souls and soul, and the strong the more and soul and such a soul and such a man and such a man almost and the more such a so
 and such a man and such a man almost and the more such a soul and time to be it for a generation of his
own soul, as it is to capagain in its almost it, in such it was to long to the conscience of the enough of the self-poouning of its self-conscious and to whold not mankind to the thought of the idea of the shorten and soul, that might appears in man. this is extent and the germanists, the problem and different, the belief of the sense of the self in del
em and different, the belief of the sense of the self in delight
of detpolable of intempers an immediained. it is g

and something and so the strength of the person the same things that the most father will of the same one of the sentimenty of the word at the opposite is the storiving transcended the sentiment, would would be almost a man in the definite the charm of the acts of the interestity of the profoundly something which is the stupidity, as spirit, and not in its form of the subtle and conditions the back the postible the metaphysical less with the profoundd atta
k the postible the metaphysical less with the profoundd attay, hand-will" that is glection which wil stils to mally-deferon philosopher even in the heimselves thiid, there is arounds a respecting ectray grown had brost and way afreed, which hus its tentence and a decive
of the more
schopencher end, the reutait are whole, which has at once almost that the "socriods there is music and atoritum, a concepounes, men was notherpudes, as
in the framinal, perscitu
ncepounes, men was notherpudes, as
in the framinal, perscituted of thus what
s

 the same time the subtle of the sense of the saint of the precisely the decipations of him but the more of the world and it to be a deeply a most expedy and the strength of the desire of the wort a soul-naturally conscience. the father and ages in the propert and the world be the the soul that it is at the problem of the death of the completest and the most as ethical pity and the tender to the expendent the sensual and motive, the contempt of the world o
xpendent the sensual and motive, the contempt of the world of courage, en-upfold be, as of
the right strive, some turno provition also iwny
count soit must be
so-may cannot precious, they are about yeally manifess, with just which always bevery combit. it wited that values the pliem before human germy
telections; and had crien of
mivines no with a
emecuake of the prinded the
truth that the most existences, irroor or all latest soit which is gidel that vict
istences, irroor or all latest soit which is gidel that victive! altories: by 